# Custom Agent

Steps:
1. Loading LLM
2. Defining Tools
3. Creating Prompt
4. Creating the Agent

## 1. Loading LLM 

Using Hugging Face and Microsoft Phi 1.5 LLM Model <br>
Steps:<br>
i) Instantiating LLM -> HuggingFaceEndpoint<br>
ii) Instantiating ChatHuggingFace() to apply chat template<br>


In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
HF_KEY = os.environ['HUGGINGFACEHUB_API_TOKEN']

In [ ]:
# Instantiating LLM
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    huggingfacehub_api_token = HF_KEY,
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    task = "text-generation",
    max_new_tokens= 512,
    top_k= 50,
    temperature= 0.1,   
)
llm

In [3]:
# Instantiating Chat Hugging Face
from langchain.schema import (HumanMessage, SystemMessage)
from langchain_community.chat_models.huggingface import ChatHuggingFace

def chat():
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content="What happens when an unstoppable force meets an immovable object?")
    ]

    chat_model = ChatHuggingFace(llm=llm)
    return chat_model

## 2. Defining Tools

A simple Python function to calculate number of vowels in a sentence.

In [4]:
from langchain.agents import tool

@tool
def get_vowels_count(sentence: str) -> int:
    """Returns the number of vowels in a sentence."""
    vowels = 'aeiouAEIOU'
    vowel_count = 0

    for char in sentence:
        if char in vowels:
            vowel_count += 1
    return vowel_count

get_vowels_count.invoke("Hello! Namaskar...")

5

In [5]:
tools = [get_vowels_count]
tools

[StructuredTool(name='get_vowels_count', description='get_vowels_count(sentence: str) -> int - Returns the number of vowels in a sentence.', args_schema=<class 'pydantic.v1.main.get_vowels_countSchema'>, func=<function get_vowels_count at 0x7f79f05d4f70>)]

## 3. Creating Prompt

2 input variables: <br>
`input` : string containing the user objective <br>
`agent_scratchpad` : sequence of messages containing previous agent tool invocations & corresponding tool outputs

In [6]:
# Open AI Method
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a powerful assistant that doesn't know current events. You think step by step..."),
#         ("user", "{input}"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ]
# )
# prompt

In [7]:
# Hugging Face Method
from langchain import hub
from langchain.tools.render import render_text_description

# Setting up ReACT style prompt
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools])
)
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "get_vowels_count: get_vowels_count(sentence: str) -> int - Returns the number of vowels in a sentence., args: {'sentence': {'title': 'Sentence', 'type': 'string'}}", 'tool_names': 'get_vowels_count'}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-json', 'lc_hub_commit_hash': '669cf4d6988c3b8994a8189edb3891e07948e1c0abfd500823914548c53afa7c'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: {tool_names}\n\nThe $JSON_BLOB should only contain a SINGLE ac

## 4. Creating Agent

2 utility functions: 
- Component for formatting intermediate steps (agent action, tool output pairs) to input messages that can be sent to the model
- Component for converting the output message into an agent action/agent finish

In [ ]:
from langchain.agents import initialize_agent, AgentType

agent1 = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
agent1

In [36]:
agent1.invoke({
    "input": "Find the number of vowels present in 'Hello, How are you?'"
})

{'input': "Find the number of vowels present in 'Hello, How are you?'",
 'output': "There are 7 vowels present in 'Hello, How are you?'"}

In [ ]:
from langchain.agents import create_react_agent

agent2 = create_react_agent(llm, tools, prompt)
agent2

In [ ]:
from langchain.agents import AgentExecutor

agent_exec = AgentExecutor(agent=agent2, 
                           tools=tools, 
                           verbose=True,
                           handle_parsing_errors="Check your output and make sure it conforms, use the Action/Action Input syntax")
agent_exec

In [44]:
agent_exec.invoke({"input": "Find the number of vowels present in 'Namaste! How are you?'\n\Action: I am unable to calculate."})



> Entering new AgentExecutor chain...
Could not parse LLM output: `Thought: I need to use the provided tool to count the number of vowels in the given sentence.
Action:
```
{
  "action": "get_vowels_count",
  "action_input": "Namaste! How are you?"
}
```
Observation`Check your output and make sure it conforms, use the Action/Action Input syntaxCould not parse LLM output: ` I need to use the provided tool to count the number of vowels in the given sentence.
Action:
```
{
  "action": "get_vowels_count",
  "action_input": "Namaste! How are you?"
}
```
Observation`Check your output and make sure it conforms, use the Action/Action Input syntaxCould not parse LLM output: ` I need to use the provided tool to count the number of vowels in the given sentence.
Action:
```
{
  "action": "get_vowels_count",
  "action_input": "Namaste! How are you?"
}
```
Observation`Check your output and make sure it conforms, use the Action/Action Input syntaxCould not parse LLM output: ` I need to use the prov

{'input': "Find the number of vowels present in 'Namaste! How are you?'\n\\Action: I am unable to calculate.",
 'output': 'I am unable to calculate the number of vowels in the given sentence as I need to use the provided tool to count the number of vowels. However, I am unable to use the tool due to an error in the parsing of my output.</s>'}

In [ ]:
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)

chat_model = chat()
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent_ = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)
agent_

In [ ]:
from langchain.agents import AgentExecutor

agent_exec = AgentExecutor(agent=agent_, 
                           tools=tools, 
                           verbose=True,
                           handle_parsing_errors="Check your output and make sure it conforms, use the Action/Action Input syntax")
agent_exec

In [50]:
response = agent_exec.invoke({"input": "Find the number of vowels present in 'Namaste! How are you?'"})
response



> Entering new AgentExecutor chain...
Could not parse LLM output: `Thought: I need to find the number of vowels in the given sentence. I can use the get_vowels_count tool for this.

Action:
```
{
  "action": "get_vowels_count",
  "action_input": "Namaste! How are you?"
}
```
Observation`Check your output and make sure it conforms, use the Action/Action Input syntaxCould not parse LLM output: `

Action:
```
{
  "action": "get_vowels_count",
  "action_input": "Namaste! How are you?"
}
```
Observation`Check your output and make sure it conforms, use the Action/Action Input syntaxCould not parse LLM output: `

Observation`Check your output and make sure it conforms, use the Action/Action Input syntaxCould not parse LLM output: `

The action I need to take is to use the get_vowels_count tool on the input sentence 'Namaste! How are you?'. I will format this as a json blob for the assistant to execute.

Action:
```
{
  "action": "get_vowels_count",
  "action_input": "Namaste! How are you?"


{'input': "Find the number of vowels present in 'Namaste! How are you?'",
 'output': "The number of vowels in 'Namaste! How are you?' is 5.</s>"}